<a href="https://colab.research.google.com/github/wynniegross1/vegetation_anomalies/blob/main/vci_calculation_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap
!pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:00


In [3]:
import geemap
import ee
from google.colab import userdata
from datetime import datetime

# INITIALIZE GEE PROJECT #
ee.Authenticate()
ee.Initialize(project=userdata.get('projectname'))

def calculate_vci(
    region_of_interest,
    start_date,
    end_date,
    source_data='MODIS/061/MOD09A1',
    mask_cropland=True,
    cloud_masking=True
):
    """
    Calculates Vegetation Condition Index (VCI) for a given region and time period.

    Args:
        region_of_interest: ee.FeatureCollection. The area to analyze.
        start_date (str): Start date for the period (YYYY-MM-DD).
        end_date (str): End date for the period (YYYY-MM-DD).
        source_data (str): The Earth Engine ImageCollection ID. Defaults to MODIS.
        mask_cropland (bool): Whether to apply a cropland mask.
        cloud_masking (bool): Whether to apply cloud masking.

    Returns:
        tuple: (ee.Image, geemap.Map) The VCI image and the map object.
    """
    # Helper functions
    def mask_clouds(image):
        """Masks clouds using the StateQA band of MODIS."""
        QA = image.select(['StateQA'])
        bitMask = 1 << 10
        return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))

    def crop_mask(image):
        """Masks non-cropland areas using ESA WorldCover."""
        esa = ee.ImageCollection('ESA/WorldCover/v100')
        esa_latest = esa.limit(1, 'system:time_start').first()
        cropland = esa_latest.updateMask(
            esa_latest.select('Map').eq(40).clip(region_of_interest)
        )
        return image.updateMask(cropland)

    def add_ndvi(image):
        """Calculates NDVI and adds it as a band."""
        ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).multiply(
            10000).toInt16()
        return image.addBands(ndvi.rename('ndvi'))

    # Load the image collection
    collection = ee.ImageCollection(source_data)

    # Filter by region of interest and date
    collection = collection.filterBounds(region_of_interest).filterDate(start_date, end_date)

    # Apply cloud masking if requested
    if cloud_masking:
        collection = collection.map(mask_clouds)

    # Calculate NDVI
    ndvi_collection = collection.map(add_ndvi).select('ndvi')

    # Apply cropland mask if requested
    if mask_cropland:
        ndvi_collection = ndvi_collection.map(crop_mask)

    # Calculate min and max NDVI
    ndvi_min = ndvi_collection.min()
    ndvi_max = ndvi_collection.max()

    # Calculate VCI
    vci = ndvi_collection.map(lambda img:
        img.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).multiply(100)
    ).mean()

    # Create a map object
    Map = geemap.Map(center=region_of_interest.geometry().centroid().getInfo()['coordinates'][::-1], zoom=7)

    # Visualization parameters
    vci_vis = {
        'min': 0,
        'max': 100,
        'palette': ['red', 'yellow', 'green']
    }

    # Add the VCI layer to the map
    Map.addLayer(vci, vci_vis, 'Vegetation Condition Index')

    # Adding Outline to the Map
    outline = region_of_interest.style(fillColor='00000000')
    Map.addLayer(outline, {}, "Region of Interest")

    # Add legend
    Map.add_colorbar(
        vci_vis, label="Vegetation Condition Index",
        layer_name="VCI", orientation="horizontal"
    )

    return vci, Map

# Example usage:
# Define Region of Interest (replace with your actual ROI)
region_of_interest = ee.FeatureCollection("projects/ee-aeubank/assets/sudan_admin1").filter(ee.Filter.Or(
    ee.Filter.eq('admin_1', 'Sennar'),
    ee.Filter.eq('admin_1', 'Aj Jazirah')
))

# Calculate VCI and get map
vci_result, vci_map = calculate_vci(
    region_of_interest,
    '2023-01-01',
    '2023-12-31'
)

# Display the map
vci_map

Map(center=[13.593742636930632, 33.73151043658074], controls=(WidgetControl(options=['position', 'transparent_…